# Reinforcement Learning Pipeline

Пайплайны для:
- DQN (Deep Q-Network)
- PPO (Proximal Policy Optimization)
- A2C/A3C
- Gym environments

In [ ]:
!pip install gym stable-baselines3 torch numpy -q

In [ ]:
import gym
import numpy as np
import torch
from stable_baselines3 import DQN, PPO, A2C
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

print("✓ Библиотеки загружены!")

## 1. Создание окружения

In [ ]:
# === ВАШЕ ОКРУЖЕНИЕ ===
# Используйте готовое окружение из Gym или создайте свое

ENV_NAME = 'CartPole-v1'  # Примеры: 'LunarLander-v2', 'Acrobot-v1', 'MountainCar-v0'

# Создание окружения
env = gym.make(ENV_NAME)

print(f"Окружение: {ENV_NAME}")
print(f"Пространство наблюдений: {env.observation_space}")
print(f"Пространство действий: {env.action_space}")

# Тестирование окружения
obs = env.reset()
print(f"\nНачальное наблюдение: {obs}")

## 2. DQN - Deep Q-Network

In [ ]:
# Создание DQN модели
dqn_model = DQN(
    'MlpPolicy',
    env,
    learning_rate=1e-3,
    buffer_size=50000,
    learning_starts=1000,
    batch_size=32,
    tau=1.0,
    gamma=0.99,
    train_freq=4,
    gradient_steps=1,
    target_update_interval=1000,
    exploration_fraction=0.1,
    exploration_initial_eps=1.0,
    exploration_final_eps=0.05,
    verbose=1,
    tensorboard_log="./dqn_tensorboard/"
)

print("✓ DQN модель создана!")

In [ ]:
# Обучение DQN
TIMESTEPS = 100000

# Callback для остановки при достижении целевой награды
callback_on_best = StopTrainingOnRewardThreshold(reward_threshold=195, verbose=1)
eval_callback = EvalCallback(
    env,
    callback_on_new_best=callback_on_best,
    eval_freq=10000,
    best_model_save_path='./logs/',
    verbose=1
)

dqn_model.learn(total_timesteps=TIMESTEPS, callback=eval_callback)

# Сохранение
dqn_model.save("dqn_model")
print("✓ DQN модель обучена!")

## 3. PPO - Proximal Policy Optimization

In [ ]:
# Векторизованное окружение для PPO
vec_env = make_vec_env(ENV_NAME, n_envs=4)

# PPO модель
ppo_model = PPO(
    'MlpPolicy',
    vec_env,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.2,
    verbose=1,
    tensorboard_log="./ppo_tensorboard/"
)

print("✓ PPO модель создана!")

In [ ]:
# Обучение PPO
ppo_model.learn(total_timesteps=TIMESTEPS)

# Сохранение
ppo_model.save("ppo_model")
print("✓ PPO модель обучена!")

## 4. Оценка моделей

In [ ]:
# Загрузка обученной модели
trained_model = DQN.load("dqn_model", env=env)
# trained_model = PPO.load("ppo_model", env=vec_env)

# Оценка
mean_reward, std_reward = evaluate_policy(
    trained_model, 
    env, 
    n_eval_episodes=10,
    deterministic=True
)

print(f"Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

## 5. Визуализация агента

In [ ]:
# Запуск обученного агента
def visualize_agent(model, env, n_episodes=5):
    for episode in range(n_episodes):
        obs = env.reset()
        done = False
        total_reward = 0
        
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, info = env.step(action)
            total_reward += reward
            env.render()  # Визуализация (если поддерживается)
        
        print(f"Episode {episode + 1}: Total Reward = {total_reward}")
    
    env.close()

# visualize_agent(trained_model, env, n_episodes=3)

## 6. Кастомное окружение

In [ ]:
# Если нужно создать свое окружение
# from gym import Env, spaces

# class CustomEnv(Env):
#     def __init__(self):
#         super(CustomEnv, self).__init__()
#         self.action_space = spaces.Discrete(2)  # Пример: 2 действия
#         self.observation_space = spaces.Box(low=0, high=1, shape=(4,))  # 4 признака
    
#     def reset(self):
#         # Инициализация начального состояния
#         return np.array([0.0, 0.0, 0.0, 0.0])
    
#     def step(self, action):
#         # Логика выполнения действия
#         observation = np.random.rand(4)
#         reward = 1.0
#         done = False
#         info = {}
#         return observation, reward, done, info
    
#     def render(self, mode='human'):
#         pass